In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-06-14 06:56:51,674: Credentials are already in use. The existing account in the session will be replaced.


In [4]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                    charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto-3g')

/tmp/ipykernel_2328/3178053827.py:5: DeprecationWarning: The PySCFDriver class is deprecated as of version 0.2.0 and will be removed no sooner than 3 months after the release. Instead use the PySCFDriver class from qiskit_nature.drivers.second_quantization.pyscfd.
  driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto-3g')


In [6]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper

In [10]:
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])

Fermionic Operator
register length=4, number terms=14
  (0.18093119978423142+0j) * ( +_0 -_1 +_2 -_3 )
+ (-0.1809311997842314+0j) * ( +_0 -_1 -_2 +_3 )
+ (-0.1809311997842314+0j) * ( -_0 +_1 +_2 -_3 )
+ (0.18093119978423142+0j) * ( -_0 +_1 -_2 +_3 )
 ...


In [11]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

0.045232799946057854 * YYYY
+ 0.045232799946057854 * XXYY
+ 0.045232799946057854 * YYXX
+ 0.045232799946057854 * XXXX
- 0.8105479805373277 * IIII
- 0.22575349222402374 * ZIII
+ 0.17218393261915543 * IZII
+ 0.12091263261776627 * ZZII
- 0.22575349222402372 * IIZI
+ 0.17464343068300453 * ZIZI
+ 0.16614543256382414 * IZZI
+ 0.17218393261915543 * IIIZ
+ 0.16614543256382414 * ZIIZ
+ 0.16892753870087907 * IZIZ
+ 0.12091263261776627 * IIZZ


In [12]:
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
print(qubit_op)

0.18093119978423142 * XX
- 1.05237324577286 * II
- 0.39793742484317907 * ZI
+ 0.397937424843179 * IZ
- 0.011280104256235324 * ZZ


In [13]:
qubit_converter = QubitConverter(mapper=BravyiKitaevMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

0.045232799946057854 * ZXIX
- 0.045232799946057854 * IXZX
- 0.045232799946057854 * ZXZX
+ 0.045232799946057854 * IXIX
- 0.8105479805373277 * IIII
- 0.22575349222402374 * ZZZI
+ 0.17218393261915543 * IZII
+ 0.12091263261776627 * ZIZI
- 0.22575349222402372 * IIZZ
+ 0.17464343068300453 * ZZIZ
+ 0.16614543256382414 * IZZZ
+ 0.17218393261915543 * IIIZ
+ 0.16614543256382414 * ZZZZ
+ 0.16892753870087907 * IZIZ
+ 0.12091263261776627 * IIZI


In [18]:
es_problem = ElectronicStructureProblem(driver)
second_q_ops = es_problem.second_q_ops()

In [21]:
main_op = second_q_ops[0]
num_particles = (es_problem.molecule_data_transformed.num_alpha,
                 es_problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * es_problem.molecule_data.num_molecular_orbitals

# mapper = JordanWignerMapper() 

mapper = ParityMapper()

converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
H2_op = converter.convert(main_op, num_particles=num_particles)
print("Qubit Hamiltonian:\n", H2_op)

Qubit Hamiltonian:
 0.18093119978423142 * XX
- 1.05237324577286 * II
- 0.39793742484317907 * ZI
+ 0.397937424843179 * IZ
- 0.011280104256235324 * ZZ
